In [0]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [0]:
# Lendo o arquivo CSV "Carros-2.csv" e criando um DataFrame temporário
# - inferSchema=True: Permite que o Spark detecte automaticamente os tipos de dados de cada coluna
# - header=True: Indica que a primeira linha do arquivo contém os nomes das colunas
# - sep=";": Define o delimitador como ponto e vírgula (comum em arquivos CSV com dados separados por ponto e vírgula)
Carros_temp = spark.read.csv("/FileStore/tables/Carros-2.csv", inferSchema=True, header=True, sep=";")

# Exibindo as primeiras 5 linhas do DataFrame "Carros_temp" para visualização dos dados carregados
Carros_temp.show(5)


+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [0]:
# Selecionando as colunas específicas do DataFrame "Carros_temp" 
# e criando um novo DataFrame "Carros" com apenas as colunas: "Consumo", "Cilindros", "Cilindradas", e "HP"
Carros = Carros_temp.select("Consumo", "Cilindros", "Cilindradas", "HP")

# Exibindo as primeiras 5 linhas do novo DataFrame "Carros" com as colunas selecionadas
Carros.show(5)


+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [0]:
# Utilizando o VectorAssembler para transformar as colunas "Consumo", "Cilindros" e "Cilindradas" em um vetor de características
# - inputCols: Define as colunas de entrada que serão combinadas em um único vetor
# - outputCol: Especifica o nome da nova coluna que conterá o vetor de características resultante
veccaracteristicas = VectorAssembler(inputCols=["Consumo", "Cilindros", "Cilindradas"], outputCol="caracteristicas")

# Aplicando a transformação ao DataFrame "Carros" e criando a coluna "caracteristicas" com o vetor resultante
Carros = veccaracteristicas.transform(Carros)

# Exibindo as primeiras 5 linhas do DataFrame "Carros" com a nova coluna "caracteristicas"
Carros.show(5)


+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [0]:
# Dividindo o DataFrame "Carros" em dois subconjuntos: treino (70%) e teste (30%)
# - randomSplit([0.7, 0.3]): Define a proporção de divisão entre os conjuntos de treino e teste
# O método randomSplit divide os dados aleatoriamente de acordo com a proporção especificada
CarrosTreino, CarrosTeste = Carros.randomSplit([0.7, 0.3])

# Exibindo a quantidade de registros no DataFrame de treino "CarrosTreino"
print(CarrosTreino.count())

# Exibindo a quantidade de registros no DataFrame de teste "CarrosTeste"
print(CarrosTeste.count())


21
11


In [0]:
# Definindo o modelo de regressão linear usando a classe LinearRegression
# - featuresCol: Especifica a coluna que contém as características de entrada (neste caso, "caracteristicas")
# - labelCol: Define a coluna que contém o valor alvo ou rótulo (neste caso, "HP")
reglin = LinearRegression(featuresCol="caracteristicas", labelCol="HP")

# Treinando o modelo com o conjunto de dados de treino "CarrosTreino"
# O modelo ajusta os parâmetros de acordo com os dados de treino
modelo = reglin.fit(CarrosTreino)


In [0]:
# Usando o modelo treinado para fazer previsões no conjunto de teste "CarrosTeste"
# A transformação aplica o modelo aos dados de teste e gera previsões para o valor da variável "HP"
previsao = modelo.transform(CarrosTeste)

# Exibindo as primeiras 5 linhas do DataFrame "previsao" com os resultados da previsão
# O DataFrame "previsao" agora inclui a coluna "prediction" com os valores previstos pelo modelo
previsao.show(5)


In [0]:
# Avaliando a performance do modelo utilizando o RegressionEvaluator
# - predictionCol: Define a coluna que contém as previsões feitas pelo modelo ("prediction")
# - labelCol: Define a coluna que contém os valores reais ("HP")
# - metricName='rmse': Define que a métrica a ser calculada é o Root Mean Squared Error (RMSE)
avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="HP", metricName='rmse')

# Calculando o RMSE (Root Mean Squared Error) no conjunto de previsões "previsao"
# O RMSE mede a diferença média entre os valores previstos e reais, quanto menor, melhor
rmse = avaliar.evaluate(previsao)

# Exibindo o valor do RMSE
print(rmse)
